https://dailyheumsi.tistory.com/116

https://4four.us/article/2017/05/gradient-boosting-simply

# GBM(Gradient Boosting Machine)

부스팅 알고리즘은 여러 개의 약한 학습기를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식.

GBM은 가중치 업데이트를 경사 하강법을 이용함.

**경사 하강법:** 반복 수행을 통해 오류를 최소화할 수 있도록 가중치 업데이트 값을 도출하는 기법

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [6]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [7]:
def get_human_dataset( ):
    feature_name_df = pd.read_csv('human_activity/features.txt',sep='\s+', header=None,names=['column_index','column_name'])

    new_feature_name_df = get_new_feature_name_df(feature_name_df)

    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()

    X_train = pd.read_csv('human_activity/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('human_activity/X_test.txt',sep='\s+', names=feature_name)

    y_train = pd.read_csv('human_activity/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('human_activity/y_test.txt',sep='\s+',header=None,names=['action'])
    
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()

In [9]:
from sklearn.metrics import accuracy_score
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train,y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print("GBM 수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

GBM 정확도: 0.9386
GBM 수행 시간: 196.9 초 


## GBM 하이퍼 파라미터 및 튜닝

- **loss:**

경사 하강법에서 사용할 비용 함수를 지정. 특별한 이유가 없으면 기본값인 'deviance'를 그대로 적용

- **learning_rate:**

GBM이 학습을 진행할 때마다 적용하는 학습률. Weak learner가 순차적으로 오류 값을 보정해 나가는 데 적용하는 계수. 0~1 사이의 값을 지정할 수 있으며 기본값은 0.1.

너무 작은 값을 적용하면 업데이트 되는 값이 작아져서 최소 오류 값을 찾아 예측 성능이 높아질 가능성이 높음. 하지만 많은 Weak learner는 순차적인 반복이 필요해서 수행 시간이 오래 걸리고, 또 너무 작게 설정하면 모든 Weak leaner의 반복이 완료돼도 최소 오류 값을 찾지 못할 수 있음. 반대로 큰 값을 적용하면 최소 오류 값을 찾지 못하고 그냥 지나쳐 버려 예측 성능이 떨어질 가능성이 높아지지만, 빠른 수행이 가능함.

learning_rate를 작게 하고 n_estimators를 크게하면 더 이상 성능이 좋아지지 않는 한계점까지 예측 성능이 조금씩 좋아질 수 있음.

하지만 수행 시간이 너무 오래 걸리는 단점이 있고, 예측 성능 역시 현격히 좋아지는 것은 아님.

- **n_estimators:**

weak learner의 개수. weak learner가 순차적으로 오류를 보정하므로 개수가 많을수록 예측 성능이 일정 수준까지는 좋아질 수 있음. 하지만 개수가 많을수록 수행 시간이 오래 걸림. 기본값은 100

- **subsample:**

weak learner가 학습에 사용하는 데이터의 샘플링 비율. 기본값은 1이며 이는 전체 학습 데이터를 기반으로 학습한다는 의미(0.5이면 학습 데이터의 50%). 과적합이 염려되는 경우 subsample을 1보다 작은 값으로 설정함.